In [1]:
import torch.nn.functional as F
from utils.utils import *
from datetime import datetime
from evaluation.train_inception_model import SpectrogramInception3
from data.preprocessing import AudioProcessor
from torch.utils.data import DataLoader
from evaluation.metrics.inception_score import InceptionScore
from data.audio_transforms import MelScale
from data.loaders import get_data_loader

In [2]:
config = {
    "model_name": "footsteps_inception_model_best_2021-04-29.pt",
    "comments": "inception trained on footsteps dataset",
    "state_dict_path": "evaluation/inception_models/footsteps_inception_model_best_2021-04-29.pt",
    
    "real_samples_path": "../_data/footsteps_real",
    "synth_samples_path": "../_data/footsteps_generated_23-04-2021_15h",
    
    "output_path": "evaluation",
    "output_folder": "evaluation_metrics",
    
    "batch_size": 20,

    "real_samples_loader_config": {
        "dbname": "footsteps",
        "data_path": "../_data/footsteps_real/",
        "criteria": {},
        "shuffle": True,
        "tr_val_split": 1.0
    },

    "synth_samples_loader_config": {
        "dbname": "footsteps",
        "data_path": "../_data/footsteps_generated_23-04-2021_15h/",
        "criteria": {},
        "shuffle": True,
        "tr_val_split": 1.0
    },
    
    "transform_config": {
        "transform": "stft",
        "fade_out": True,
        "fft_size": 1024,
        "win_size": 1024,
        "n_frames": 64,
        "hop_size": 256,
        "log": False,
        "ifreq": False,
        "sample_rate": 16000,
        "audio_length": 16000
    }
}

In [3]:
model_name = config['model_name']
state_dict_path = config['state_dict_path']
output_path = mkdir_in_path(config['output_path'], config['output_folder'])

In [4]:
real_samples_loader_config = config['real_samples_loader_config']

transform_config = config['transform_config']
transform = transform_config['transform']

dbname = real_samples_loader_config['dbname']

batch_size = config['batch_size']

processor = AudioProcessor(**transform_config)

loader_module = get_data_loader(dbname)

real_samples_loader = loader_module(name=dbname + '_' + transform, preprocessing=processor, **real_samples_loader_config)

n_real_samples = len(real_samples_loader)
print('n_real_samples: ', n_real_samples)

real_samples_data_loader = DataLoader(real_samples_loader,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        num_workers=2)

device = 'cuda' if GPU_is_available() else 'cpu'

state_dict = torch.load(state_dict_path, map_location=device)
inception_footsteps = SpectrogramInception3(state_dict['fc.weight'].shape[0], aux_logits=False)
inception_footsteps.load_state_dict(state_dict)
inception_footsteps = inception_footsteps.to(device)

mel = MelScale(sample_rate=transform_config['sample_rate'],
                fft_size=transform_config['fft_size'],
                n_mel=transform_config.get('n_mel', 256),
                rm_dc=True)
mel = mel.to(device)

Configuring stft transform...
Dataset ../_data/footsteps_real/_processed/footsteps_stft/footsteps_stft.pt exists. Reloading...
n_real_samples:  720
Cuda not available. Running on CPU


In [5]:
is_maker_real_samples = InceptionScore()
inception_score_real_samples = []

with torch.no_grad():
    for batch_idx, data in enumerate(real_samples_data_loader):
        input, labels = data
        input.to(device)
        input = mel(input.float())
        mag_input = F.interpolate(input[:, 0:1], (299, 299))
        
        preds = inception_footsteps(mag_input.float())
        
        is_maker_real_samples.updateWithMiniBatch(preds)
        inception_score_real_samples.append(is_maker_real_samples.getScore())
        
        print('batch: ', batch_idx, 'IS: ', is_maker_real_samples.getScore())

batch:  0 IS:  5.515297653349979
batch:  1 IS:  5.641838672578491
batch:  2 IS:  5.657102397558225
batch:  3 IS:  5.375245202624796
batch:  4 IS:  5.288231550886117
batch:  5 IS:  5.380001128490844
batch:  6 IS:  5.469275808491983
batch:  7 IS:  5.539409312991891
batch:  8 IS:  5.527735638695315
batch:  9 IS:  5.517051189876037
batch:  10 IS:  5.525369024649866
batch:  11 IS:  5.477996651795148
batch:  12 IS:  5.5152416326795635
batch:  13 IS:  5.538718729620946
batch:  14 IS:  5.559932113906793
batch:  15 IS:  5.555472121725741
batch:  16 IS:  5.581526436990219
batch:  17 IS:  5.581855660397229
batch:  18 IS:  5.577216771134922
batch:  19 IS:  5.585431449981089
batch:  20 IS:  5.587121140888964
batch:  21 IS:  5.599942120461706
batch:  22 IS:  5.618792129252534
batch:  23 IS:  5.612908137710384
batch:  24 IS:  5.625282067890732
batch:  25 IS:  5.622807505579048
batch:  26 IS:  5.621855821702069
batch:  27 IS:  5.630199150741367
batch:  28 IS:  5.636570104364572
batch:  29 IS:  5.64921

In [6]:
IS_mean = np.mean(inception_score_real_samples)
IS_std = np.std(inception_score_real_samples)
output_file = f'{output_path}/IS_real_{str(n_real_samples)}_{model_name}_{datetime.now().strftime("%d-%m-%y_%H_%M")}.txt'

with open(output_file, 'w') as f:
    f.write(str(IS_mean) + '\n')
    f.write(str(IS_std))
    f.close()

In [7]:
synth_samples_loader_config = config['synth_samples_loader_config']
synth_samples_loader = loader_module(name=dbname + '_' + transform, preprocessing=processor, **synth_samples_loader_config)

n_synth_samples = len(synth_samples_loader)
print('n_synth_samples: ', n_synth_samples)

synth_samples_data_loader = DataLoader(synth_samples_loader,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        num_workers=2)

Dataset ../_data/footsteps_generated_23-04-2021_15h/_processed/footsteps_stft/footsteps_stft.pt exists. Reloading...
n_synth_samples:  600


In [8]:
is_maker_synth_samples = InceptionScore()
inception_score_synth_samples = []

with torch.no_grad():
    for batch_idx, data in enumerate(synth_samples_data_loader):
        input, labels = data
        input.to(device)
        input = mel(input.float())
        mag_input = F.interpolate(input[:, 0:1], (299, 299))
        
        preds = inception_footsteps(mag_input.float())
        
        is_maker_synth_samples.updateWithMiniBatch(preds)
        inception_score_synth_samples.append(is_maker_synth_samples.getScore())
        
        print('batch: ', batch_idx, 'IS: ', is_maker_synth_samples.getScore())

batch:  0 IS:  4.315168870031892
batch:  1 IS:  4.694407361890251
batch:  2 IS:  4.6436985220799
batch:  3 IS:  4.689670049925526
batch:  4 IS:  4.771269784044424
batch:  5 IS:  4.815543565590214
batch:  6 IS:  4.765092178648397
batch:  7 IS:  4.796038717878455
batch:  8 IS:  4.8390386806536245
batch:  9 IS:  4.739858182600099
batch:  10 IS:  4.734211219417201
batch:  11 IS:  4.755173327229082
batch:  12 IS:  4.739344396190907
batch:  13 IS:  4.76350739057944
batch:  14 IS:  4.756118606816763
batch:  15 IS:  4.738045434727046
batch:  16 IS:  4.753992754371582
batch:  17 IS:  4.765947670416352
batch:  18 IS:  4.7886794725697905
batch:  19 IS:  4.8153030041814
batch:  20 IS:  4.829124409185444
batch:  21 IS:  4.8363358387180675
batch:  22 IS:  4.803916798306491
batch:  23 IS:  4.79672840772187
batch:  24 IS:  4.786433839936384
batch:  25 IS:  4.769429286304862
batch:  26 IS:  4.772376556260819
batch:  27 IS:  4.7522725003721495
batch:  28 IS:  4.754220470030034
batch:  29 IS:  4.77207198

In [9]:
IS_mean = np.mean(inception_score_synth_samples)
IS_std = np.std(inception_score_synth_samples)
output_file = f'{output_path}/IS_synth_{str(n_synth_samples)}_{model_name}_{datetime.now().strftime("%d-%m-%y_%H_%M")}.txt'

with open(output_file, 'w') as f:
    f.write(str(IS_mean) + '\n')
    f.write(str(IS_std))
    f.close()